In [1]:
from torch.optim import lr_scheduler
from torch import nn
from calrank import *
from video_utils import *

Setting Parameters

In [2]:
f_type = 'gamma_nuclear' # options are: [gamma_nuclear | LNN | Logarithm | ETP | Geman | Laplace]
lr = 200
lam=10000
f_gamma = 500
f_lam = 1
step = 500
use_f = False

num_inv = 200
num_g = 10000
n = 5000
grad_mask = True
update_lr = True
lr_change_step = 400
lr_change_rate = 0.5

Training Process

In [3]:
video_input_path = r'escalator.avi'
video_output_path = r'escalator_out.avi'

# read the video and convert it into a N*W*H*C numpy array
video_np_array = video_to_numpy_array(video_input_path)
print('original shape:', video_np_array.shape)
# only use first 100 frames, due to memory reason
video_np_array = video_np_array[:100, :, :]
print('truncat shape:', video_np_array.shape)
input = video_np_array[...,0]
frame_num = input.shape[0]
input = input.reshape(frame_num,-1)
input = video_np_array[...,0]
frame_num = input.shape[0]
input = input.reshape(frame_num,-1)
if use_f:
    input = input/255.0
print('input shape:', input.shape)


gpu = try_gpu(0)
target = torch.tensor(input,requires_grad=False,dtype=torch.float32,device=gpu)
w = torch.tensor(input,requires_grad=True,dtype=torch.float32,device=gpu)
loss = nn.L1Loss(reduction='none')
print("w.device:",w.device)
print("w.shape",w.shape)
# gamma_nuclear | LNN | Logarithm | ETP | Geman | Laplace
if f_type == 'gamma_nuclear':
    parameters = parameters_gamma_nuclear_norm(gamma=f_gamma,lam=f_lam)
elif f_type == 'LNN':
    parameters = parameters_LNN(lam=f_lam)
elif f_type == 'Geman':
    parameters = parameters_Geman(gamma=f_gamma,lam=f_lam)
elif f_type == 'Laplace':
    parameters = parameters_Laplace(gamma=f_gamma,lam=f_lam)
elif f_type == 'ETP':
    parameters = parameters_ETP(gamma=f_gamma,lam=f_lam)

optimizer = torch.optim.Adam([w], lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=lr_change_step, gamma=lr_change_rate)
for i in range(step):
    optimizer.zero_grad()
    l1 = (loss(target, w)).sum()
    non_zero_elements = input.shape[0] * input.shape[1]
    l1 = lam * l1 / non_zero_elements
    if use_f:
        l2 = cal_rank_f(w,parameters=parameters,num_inv=num_inv,num_g=num_g,gpu=gpu,n=n)
    else:
        l2 = cal_rank_nuclear(w,num_inv=num_inv,num_g=num_g,gpu=gpu,n=n)
    l = l1 + l2
    if i%20==0:
        print(i,"l1:",l1.data,"l2:",l2.data,"l:",l.data)
    l.backward()
    optimizer.step()
    if update_lr:
        scheduler.step()

original shape: (198, 130, 160, 3)
truncat shape: (100, 130, 160, 3)
input shape: (100, 20800)
w.device: cuda:0
w.shape torch.Size([100, 20800])
0 l1: tensor(0., device='cuda:0') l2: tensor(512781.5000, device='cuda:0') l: tensor(512781.5000, device='cuda:0')
20 l1: tensor(287168.2812, device='cuda:0') l2: tensor(635059.5625, device='cuda:0') l: tensor(922227.8750, device='cuda:0')
40 l1: tensor(172673., device='cuda:0') l2: tensor(472439.5000, device='cuda:0') l: tensor(645112.5000, device='cuda:0')
60 l1: tensor(152495.1406, device='cuda:0') l2: tensor(451643.3750, device='cuda:0') l: tensor(604138.5000, device='cuda:0')
80 l1: tensor(149562.1875, device='cuda:0') l2: tensor(448195.4062, device='cuda:0') l: tensor(597757.6250, device='cuda:0')
100 l1: tensor(150569.1562, device='cuda:0') l2: tensor(442706.1562, device='cuda:0') l: tensor(593275.3125, device='cuda:0')
120 l1: tensor(151499.5625, device='cuda:0') l2: tensor(440883.0938, device='cuda:0') l: tensor(592382.6250, device='c

Saving results

In [4]:
save_video(video_np_array, w.cpu().detach().numpy(), "", '%s_result.avi'%('escalator'), use_f)
save_video(video_np_array, (target-w).cpu().detach().numpy(), "", '%s_residual_result.avi'%('escalator'),residual=True, use_f=use_f)